In [1]:
from parse_articles import parse_articles_from_url
from collections import defaultdict
import logging
from gensim import corpora, models, similarities

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# get the data
articles = parse_articles_from_url('https://hackathon17.mope.ml/HackathonSite/News300.txt')
#context = parse_articles_from_url('https://hackathon17.mope.ml/HackathonSite/NewsAll.txt')

#print(articles[0])

c:\users\patrick\appdata\local\programs\python\python36-32\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# load text
from textblob import TextBlob

texts = list()
for article in articles:
    blob = TextBlob(article['headLine'] + " " + article['story'])
    #texts = blob.words.singularize()
    phrases = blob.noun_phrases
    thisText = list()
    for phrase in phrases:
        thisText.append(phrase)
    texts.append(thisText)
    
#print(texts)
        
    #ngrams = blob.ngrams(n=2)
    #for ngram in ngrams:
    #    phrase = ""
    #    for word in ngram:
    #        phrase = phrase + " " + word
    #    texts.append(phrase.strip())

print("ok")
    

ok


In [3]:
# intitialize stoplists
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

stoplist += ['fitch', 'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december', 'said', 'would']

#print(stoplist)
#print("------------------------------")
#print(texts)

newTexts = list()
for text in texts:
    newText = list()
    for word in text:
        if word not in stoplist:
            newText.append(word)
    newTexts.append(newText)
texts = newTexts

#print("------------------------------")
#print(texts)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Patrick\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]
#print(texts[0])


In [5]:
# create dictionary
#print(texts)
dictionary = corpora.Dictionary(texts)
dictionary.save('patrick/dict.txt')  # store the dictionary, for future reference

#print(dictionary)

2017-11-25 11:42:00,121 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-11-25 11:42:00,151 : INFO : built Dictionary(2432 unique tokens: ['bangladesh', 'york fed', 'federal reserve bank', 'york', 'fed']...) from 300 documents (total 10739 corpus positions)
2017-11-25 11:42:00,155 : INFO : saving Dictionary object under patrick/dict.txt, separately None
2017-11-25 11:42:00,176 : INFO : saved patrick/dict.txt


In [6]:
# create corpus
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('patrick/corpus.txt', corpus)  # store to disk, for later use

#print(corpus[0])

#for c in corpus:
#    print("---")
#    print(c)

2017-11-25 11:42:00,224 : INFO : storing corpus in Matrix Market format to patrick/corpus.txt
2017-11-25 11:42:00,232 : INFO : saving sparse matrix to patrick/corpus.txt
2017-11-25 11:42:00,233 : INFO : PROGRESS: saving document #0
2017-11-25 11:42:00,261 : INFO : saved 300x2432 matrix, density=0.855% (6240/729600)
2017-11-25 11:42:00,265 : INFO : saving MmCorpus index to patrick/corpus.txt.index


In [7]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus] 

weights = [0 for key in dictionary]

for doc in corpus_tfidf:
    for word in doc:
        weights[word[0]] = weights[word[0]] + word[1]

wordWeights = list()
i = 0
for w in weights:
    wordWeights.append((w, dictionary[i]))
    i += 1

wordWeights = sorted(wordWeights, key=lambda wordWeight: wordWeight[0], reverse=True)
print(wordWeights)

2017-11-25 11:42:00,313 : INFO : collecting document frequencies
2017-11-25 11:42:00,315 : INFO : PROGRESS: processing document #0
2017-11-25 11:42:00,321 : INFO : calculating IDF weights for 300 documents and 2431 features (6240 matrix non-zeros)


[(8.154218298683386, 'u.s.'), (4.233313586538496, 'china'), (3.926603879709678, 'syria'), (3.840745678583578, 'greece'), (3.7154837682389035, 'russia'), (3.6567583473414986, 'brazil'), (3.472172851526299, 'iran'), (3.312855907609076, 'york'), (3.246838873744805, 'eu'), (3.0841780378536634, 'canada'), (2.979631605803703, 'islamic'), (2.68703256444242, 'update'), (2.587576619148781, 'c $'), (2.538123340027651, 'fed'), (2.4855181135253925, 'japan'), (2.4696864058080874, 'reuters'), (2.44552925319678, 'oc'), (2.444880010173951, 'trump'), (2.433441601592205, 'duterte'), (2.412408322972797, 'italy'), (2.4039677218661133, 'europe'), (2.363675632245879, 'carolina'), (2.3488993719842215, 'oil prices'), (2.293392518933399, 'interest rates'), (2.218526845228926, 'africa'), (2.2095355244946977, 'britain'), (2.1926512707608445, 'idr'), (2.192621447880798, 'central bank'), (2.1841683033337955, 'syrian'), (2.1201037423416715, 'republican'), (2.108666815713731, 'france'), (2.084000037615353, 'gmt'), (